In [1]:
from dash import Dash, dcc, html
from dash.dependencies import Input, Output, State
import plotly.graph_objects as go
import pandas as pd
import numpy as np

In [2]:
# Initialize the Dash app
app = Dash(__name__)

def load_data():
    # Load the data
    df = pd.read_csv('geocoded_addresses.csv')
    df = df.drop(columns=['hist_distance'])
    df = df.rename(columns={'distance': 'straightline_distance'})
    df = df[df['lat'].notnull() & df['lon'].notnull()]
    return df

def bin_data(df, bin_size=0.1):
    # Create bins for latitude and longitude based on the bin size
    df['lat_bin'] = np.round(df['lat'] / bin_size) * bin_size
    df['lon_bin'] = np.round(df['lon'] / bin_size) * bin_size

    # Create a cluster label based on the bin
    df['cluster'] = df['lat_bin'].astype(str) + ',' + df['lon_bin'].astype(str)

    return df

def create_cluster_df(df):
    # Group by cluster and calculate the mean latitude and longitude
    cluster_df = df.groupby('cluster')[['lat', 'lon']].mean().reset_index()

    return cluster_df

df = load_data()
df=bin_data(df)
cluster_df = create_cluster_df(df)

app.layout = html.Div([
    dcc.Store(id='df_store', data=df.to_json(orient='split')),
    dcc.Input(id='search_bar', type='text', placeholder='Enter a street name'),
    dcc.Dropdown(id='state_filter', options=[{'label': i, 'value': i} for i in df['state'].unique()], placeholder='Filter by state'),
    dcc.Dropdown(id='type_filter', options=[{'label': i, 'value': i} for i in df['type'].unique()], placeholder='Filter by type'),
    dcc.Graph(id='map'),
    dcc.Markdown(id='clickdata', style={'backgroundColor': 'white'})
])

@app.callback(
    Output('map', 'figure'),
    [Input('search_bar', 'value'),
     Input('state_filter', 'value'),
     Input('type_filter', 'value')]
)
def update_map(search_value, state_value, type_value):
    try:
        filtered_df = filter_data(search_value, state_value, type_value)
        fig = create_map(filtered_df)
        return fig
    except Exception as e:
        return html.Div(['There was an error processing your request: {}'.format(e)])

def filter_data(search_value, state_value, type_value):
    if search_value is None:
        # If the search bar is empty, display all data
        filtered_df = df
    else:
        # If the search bar is not empty, filter the data
        filtered_df = df[df['street1'].str.contains(search_value, case=False, na=False)]

    # Apply additional filters if values are selected
    if state_value is not None:
        filtered_df = filtered_df[filtered_df['state'] == state_value]
    if type_value is not None:
        filtered_df = filtered_df[filtered_df['type'] == type_value]
    
    return filtered_df

def create_map(cluster_df):
    fig = go.Figure()

    fig.add_trace(go.Scattermapbox(
        lat=cluster_df['lat'],
        lon=cluster_df['lon'],
        mode='markers',
        marker=go.scattermapbox.Marker(
            size=10,
            color=np.random.rand(len(cluster_df), 3)  # Set a random color for each cluster
        ),
        hoverinfo='none',
        customdata=cluster_df['cluster'],  # Add the cluster label as custom data
    ))

    fig.update_layout(
        mapbox_style="open-street-map",
        mapbox=dict(
            center=dict(
                lat=46.8721, 
                lon=-113.9940
            ),
            zoom=12
        ),
        margin={"r":0,"t":0,"l":0,"b":0},
        showlegend=False,  # Show the legend
        legend=dict(y=0.5)
    )

    return fig

# Define the callback to update the clickdata div
@app.callback(
    Output('clickdata', 'children'),
    [Input('map', 'clickData')]
)
def display_click_data(clickData):
    if clickData is None:
        return "Click on a point in the map to see more information."
    else:
        # Extract the cluster from the clicked point's custom data
        cluster = clickData['points'][0]['customdata']
        # # Retrieve all rows from the original DataFrame with the same cluster
        # rows = df[df['cluster'] == cluster]
        # # Convert the rows to a DataFrame and then to a markdown table
        # markdown_table = rows.to_markdown()
        return cluster

In [3]:
# Run the app
app.run_server(mode='jupyterlab')